In [ ]:
import sys
import os
sys.path.append(os.pardir)

from dataset.mnist import load_mnist

# 训练图片，训练标签    测试图片，测试标签
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

x_train.shape, t_train.shape, x_test.shape, t_test.shape

In [ ]:
from PIL import Image
import numpy as np

def show_img(img: np.matrix):
    pil_img = Image.fromarray(np.uint8(img))
    display(pil_img)

print(t_train[0])
show_img(x_train[0].reshape(28,28))

In [54]:
import pickle

def sigmoid_function(x: np.ndarray):
    y = 1/ (np.exp(-x) + 1)
    return y

def softmax_function(x: np.array):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 溢出对策
    return np.exp(x) / np.sum(np.exp(x))

def get_test_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open('sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid_function(a1)
    
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid_function(a2)
    
    a3 = np.dot(z2, W3) + b3
    y = softmax_function(a3)
    return y

x, t = get_test_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    output_label = np.argmax(y)
    if output_label == t[i]:
        accuracy_cnt += 1

accuracy_cnt / len(x)

0.9352

## batch processing

In [55]:
x, t = get_test_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i: i+batch_size]
    y_batch = predict(network, x_batch)
    output_labels = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(output_labels == t[i:i+batch_size])

accuracy_cnt / len(x)

0.9352